<a href="https://colab.research.google.com/github/aviadm24/aviad2/blob/master/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import json
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
from tensorflow.keras.optimizers import Adam

# Load pre-trained GPT2 model and tokenizer
model = TFGPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Load the JSON data
with open('test_data.json', 'r') as f:
    data = json.load(f)

# Extract the test cases from the loaded JSON data
test_cases = data["test_cases"]

# Prepare the dataset for fine-tuning
requirements = [case["test_requirement"] for case in test_cases]
steps = [case["steps"] for case in test_cases]  # Steps remain as an array of strings

# Tokenize the test requirements (input) and steps (output)
input_encodings = tokenizer(requirements, padding=True, truncation=True, return_tensors="tf")
output_encodings = tokenizer(['\n'.join(step) for step in steps], padding=True, truncation=True, return_tensors="tf")

input_ids = input_encodings['input_ids']
output_ids = output_encodings['input_ids']

# Fine-tune the pre-trained GPT-2 model on your custom data
model.compile(optimizer=Adam(learning_rate=5e-5))

# Fine-tune the model for 3 epochs (or more depending on the size of your data)
model.fit(input_ids, output_ids, epochs=3, batch_size=8)

# Function to generate test steps in JSON format
def generate_test_steps_in_json(test_requirement):
    # Tokenize the input test requirement
    input_ids = tokenizer(test_requirement, return_tensors="tf")['input_ids']

    # Generate the test steps using the fine-tuned model
    generated_ids = model.generate(input_ids, max_length=100, num_return_sequences=1)

    # Decode the generated sequence back into readable text
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    # Split the generated text into individual steps based on newline
    generated_steps = generated_text.split('\n')

    # Create a dictionary to hold the test steps in JSON format
    test_steps_json = {
        "test_requirement": test_requirement,
        "steps": [step.strip() for step in generated_steps if step.strip()]
    }

    # Convert the dictionary to JSON string (pretty-printing for readability)
    test_steps_json_str = json.dumps(test_steps_json, indent=4)
    return test_steps_json_str

# Example test requirement
test_requirement = "Test aircraft performance during high-speed flight."

# Generate the corresponding test steps in JSON format
generated_steps_json = generate_test_steps_in_json(test_requirement)

# Print the generated test steps as JSON
print(generated_steps_json)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

FileNotFoundError: [Errno 2] No such file or directory: 'test_data.json'